# LSEG's SFI Snowflake Datawarehouse: A use-case evaluating public companies' Climate track record using Python and SQL

In this article, we will investigate Public Companies' Climate targets and whether they are on track to meet them. The data needed for such an investigation is not onboarded on [LSEG's Data Platform](https://developers.lseg.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-platform-apis) yet; but, thankfully, it is on LSEG's Sustainable Finance Investing (SFI) Snowflake.

LSEG collects data from a range of sources; for climate data, we rely on our analysts to retrieve the information directly from companies’ disclosures. The most exciting data onboarded has, recently, related to ESG and Climate topics. From popular demand, we distribute Sustainable Finance and Investing (SFI) data via the Datawarehouse known as [Snowflake](https://docs.snowflake.com/). SFI Snowflake can be accessed via [Snowflake's online platform](https://app.snowflake.com/) or simply using Python via Snowflake's API wrapper named [Snowpark](https://docs.snowflake.com/en/developer-guide/snowpark/python/index). Datasets available include ESG, Climate, EU Taxonomy and Green Revenues data. Snowflake's API is a [REST API](https://blog.postman.com/rest-api-examples/), just like LSEG's Data Platform; similarly, Snowflake created a module to wrap their API into the Python (as well as Scala and Java) module `snowpark` (in their Python library `snowflake`), just as we wrapped LSEG's Data Platform REST API into [LSEG's Data Library](https://developers.lseg.com/en/api-catalog/lseg-data-platform/lseg-data-library-for-python) for Python.

For more information on the data available on LSEG's SFI Snowflake, please look into [myaccount.lseg.com/en/product/quantitative-analytics](https://myaccount.lseg.com/en/product/quantitative-analytics). Here you will find all the related documentation necessary to understand the data-model we are leveraging. Please be aware that SFI is a set of data the SFI datasets are surfaced via the QA Quantitative Analytics (QA) product; LSEG collates several sets of data in tables in QA; some are packaged commercially-speaking under the name of 'Datastream', others under the name 'Worldscope', among these many packages, we are only interested in 'SFI' which is easy to access using Snowflake. When logging into [myaccount.lseg.com/en/product/quantitative-analytics](https://myaccount.lseg.com/en/product/quantitative-analytics), you can access PDF documentation files:

![Image](./Docs/myaccount.lseg.com.quantitative-analytics.png)

Personally, I'm more of a fan of accessing such information programmatically. In this article, I'll take you through how one can do just that.

## Beginner's steps: SFI Snowflake using Snowpark to collect ESG data

We first need to import the `snowflake-snowpark-python` library. This library allows us to connect to Snowflake's services without having to deal with things like authentication tokens ourselves:

In [ ]:
## pip install snowflake-snowpark-python
## pip install "snowflake-connector-python[pandas]"
from snowflake import snowpark
import pandas as pd
import os # This is a native library that will allow us to collect authentication details from the machine where we're running our code.
import plotly.graph_objects as go # `plotly` and its modules are useful for us to plot graphs
import numpy as np # `numpy` in this case will be useful for us to plot graphs

In the cell below, we create the global variable `g_db_session` in which we will insert our authentication details. This will enable us to envoke it like an object that we can use to access SFI data. To "insert our authentication details", we will create the `get_session` function:

In [3]:
# Global variable
g_db_session = None

def get_session(user, password):
    global g_db_session
    
    # Establish a connection and assign it to the global session variable
    connection_parameters = { 
        "account": "rddpreprod1.east-us-2.azure",
        "user": user,
        "password": password,
        "warehouse": "SFI_READER_S_WH", 
        "database": "SHAREDB_SFI_MVP",
        "role": "SNO_SNOREFDBD_SFI_READER_ROLE",
        "schema": "DBO",
        "authenticator": "externalbrowser"
    }
    
    g_db_session = snowpark.Session.builder.configs(connection_parameters).create() 

Let's pick an Organisation's Permanent IDentifier (OrgPermID) at random for now:

In [4]:
OrgPermID = "4295875633"

Note that we don't want to show our passwords, or enter it anywhere viewable to anyone reading our code. Therefore, instead of entering our password in a string, we'll create a session in the global `g_db_session` object that opens a browser window where you can enter your credentials and log into Snowflake using your password:

In [5]:
get_session("john.doe@domain.com", os.getenv('SSO_PASSWD'))

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/287e9f0e-91ec-4cf0-b7a4-c63898072181/saml2?SAMLRequest=nZJLc9sgFIX%2FioauJQHW2BJjO%2BNEdqtOYrt%2BdZIdlpDDBIEKKIr%2FfZAfnXSRLLpj4Nz7He65w5u3SnivTBuu5AigAAKPyVwVXB5GYLuZ%2BTHwjKWyoEJJNgJHZsDNeGhoJWoyaeyzXLE%2FDTPWc42kId3DCDRaEkUNN0TSihlic7KePNwTHEBCjWHaOhy4lBSGO9aztTUJw7Ztg7YXKH0IMYQwhEnoVJ3kG%2FiAqL9m1FpZlStxLXlzf%2FoEgUIYdQincITlpfCWy%2FMIvqLszyJDfmw2S3%2B5WG%2BAN7n%2B7k5J01RMr5l%2B5Tnbru7PBoxzIAw7%2BOv5YjWdpbfp5Gk73aXTXWCkaktBX1iuqrqxrnXgTmHJilCoA3cDy9IRqF94ke1XBzuYfVdRkaVxO0P9JDE%2FIe418aKZ95a%2Fxa%2FpIzomD49ZDrzdNV7cxZsZ07BMdqFadwVx5CPsw2SDIMERQTDo9%2FET8FIXKpfUnir%2FOu98BBXPtTKqtEoKLtnJJY4HLCkh8xPEcj%2FKS%2BjvBzTy834vTmI4wChGYRcdBuf1IScjevxfQxmGH1tc1nHuEsrSpRI8P3ozpStqPw8QBeh0wwu%2FPEkJqygXk6LQzBgXpBCqvd

Now we can send out SQL query:

In [6]:
# Retrieve the data from Snowflake...
query_esg = f"""
    SELECT S.FY, S.SCORECALCDT, S.SCORE, I.TITLE
    FROM ESG2SCORES S 
    JOIN ESG2ITEM I ON S.ITEM = I.ITEM 
    WHERE S.ORGPERMID = {OrgPermID} and I.ITEM IN (2,4,5,6,7,8,9,10,11,12,13,14,15,16,65)
    """

# Execute the query and get the DataFrame
df_esg = g_db_session.sql(query_esg).toPandas()

display(df_esg)

,FY,SCORECALCDT,SCORE,TITLE
0,2008,2022-08-03 10:12:43.353,0.950658,Workforce Score
1,2008,2022-08-03 10:12:43.353,0.900794,Emissions Score
2,2008,2022-08-03 10:12:43.353,0.977876,Resource Use Score
3,2008,2020-12-07 08:49:58.147,0.885417,Management Score
4,2008,2022-08-03 10:12:43.353,0.898116,Social Pillar Score
...,...,...,...,...
303,2017,2024-02-03 06:01:10.670,0.819178,Governance Pillar Score
304,2017,2024-04-06 01:19:13.830,0.526829,ESG Combined Score
305,2017,2024-04-06 01:19:13.830,0.891304,Product Responsibility Score
306,2017,2024-01-06 05:10:37.967,0.898148,CSR Strategy Score


In [7]:
# Pivot the DataFrame to get the desired format
pivot_df_esg = df_esg.pivot(index='FY', columns='TITLE', values='SCORE')

# Return most recent FY
pivot_df_esg = pivot_df_esg.iloc[[-1]].reset_index(drop=True)
pivot_df_esg.columns.name = None

display(pivot_df_esg)

,CSR Strategy Score,Community Score,ESG Combined Score,Emissions Score,Environment Pillar Score,Environmental Innovation Score,Governance Pillar Score,Human Rights Score,Management Score,Product Responsibility Score,Resource Use Score,Shareholders Score,Social Pillar Score,Workforce Score
0,0.95,0.968165,0.481272,0.806513,0.827386,0.761364,0.84,0.909524,0.956,0.787698,0.895062,0.38,0.925768,0.990637


## Intermediary steps: SFI Snowflake using the browser to collect Company data

Above, we picked a random OrgPermID, but how company is this related to? As it turns out, finding the answer to this question using SFI and SQL code allows us to go quite far.

When you open your Snowflake homepage, you ought to be greeted with something looking like:

![Image](./Docs/SFBrowsr1.PNG)

The main difference should be that you have no ongoing projects.

What we're trying to do here is to find the place where company information lies, then maybe we can find out which company has OrgID '4295875633'. To do this, let's 1st select our Role (for more info on Roles, please read [Snowflake's documentaiton](https://docs.snowflake.com/en/user-guide/security-access-control-overview)):

![Image](./Docs/SFBrowsr2.PNG)

Now that we have selected our role, let's dig through the databases we have access to:

![Image](./Docs/SFBrowsr3.PNG)

In this article, we focus on SFI databases available on Snowflake, specifically "SHAREDB_SFI_MVP" & "SFI_MASTER_MVP":

![Image](./Docs/SFBrowsr4.PNG)

I would always advise starting with the SHAREDB_SFI_MVP.DBO.QASchTableInfo table (i.e.: the SHAREDB_SFI_MVP parent database, the DBO child database, the QASchTableInfo table):

![Image](./Docs/SFBrowsr5.PNG)

![Image](./Docs/SFBrowsr6.PNG)

We can see what data lives in this table with the 'Open in Worksheet' option:

![Image](./Docs/SFBrowsr7.PNG)

We, unfortunately have to pick our role again:

![Image](./Docs/SFBrowsr8.PNG)

I would advise changing the code too as we don't have permission to change any of the data in tables. I go for

`SELECT TOP 10 * FROM SHAREDB_SFI_MVP.DBO.QASCHFIELDINFO`

![Image](./Docs/SFBrowsr9.PNG)

Let's see what company is linked with this OrgPermID '4295875633'. To do so, we'll need to venture in the world of Python and Snowflake Worksheets (later, I'll show you how you can do the same with your own Python instance). Open a new Python Worksheet:

![Image](./Docs/PythonWrkSht1.png)

Let's run the code we had above, but in Snowflake's Worksheets:

```
# The Snowpark package is required for Python Worksheets. 
# You can add more packages by selecting them using the Packages control and then importing them.
import snowflake.snowpark as snowpark
from snowflake.snowpark.functions import col
import pandas as pd

def main(session: snowpark.Session): 

    OrgPermID = "4295875633"
    
    query = f"""
    SELECT TABLE_NAME
    FROM SHAREDB_SFI_MVP.INFORMATION_SCHEMA.TABLES
    WHERE TABLE_NAME ILIKE '%Perm%'
    """
    df = session.sql(query).toPandas()
    
    print(df)
```

![Image](./Docs/PythonWrkSht2.png)

- In line 9, `OrgPermID = "4295875633"` precised the company for which we were doing our search.
- Line 11 through to 15 was our SQL query which we will send to Snowflake to run as if we were simply in a SQL Worksheet as before. This code selects the name of all the tables that can be found in the SHAREDB_SFI_MVP database.
- Line 16's `session.sql(query).toPandas()` uses the session created in lines 1 to 8 to send the `query` and transforms it into a pandas dataframe. Pandas dataframes are easy to manipulate in Python, although you can choose other types from which to work. We output this dataframe in the object `df`.
- In line 18, we simply `print` this dataframe so we can see it in the Output window below the code window in Snowflake's browser window.

You may wonder "why did we create python code simply just to send SQL code to Snowflake, when we could do this directly in Snowflake SQL Worksheets?". Great question; the reason is that we can't use loops in SQL Worksheets! Whereas in Python Worksheets, now we can use the table names to search for the ORGPERMID '4295875633' by using the below in the `main` function:

```
    query = f"""
        SELECT TABLE_NAME
        FROM SHAREDB_SFI_MVP.INFORMATION_SCHEMA.TABLES
        WHERE TABLE_NAME ILIKE '%Perm%'
        """
    df = session.sql(query).toPandas()
    
    for table in df.TABLE_NAME:
        try:
            query = f"""
                SELECT * FROM SHAREDB_SFI_MVP.DBO.{table}
                WHERE ORGPERMID = '4295875633'
                """
            _df = session.sql(query).toPandas()
            if len(_df) > 0:
                print("\n")
                print(f"{table}")
                print(_df.T)
        except:
            print("\n")
            print(f"{table} didn't have a 'ORGPERMID' column")
```

![Image](./Docs/PythonWrkSht3.png)

- In line 9 to 13, we create our SQL query, embedding our OrgPermID "4295875633", selecting the table names that were outputed before, all to have this list of tables in the dataframe `df` on line 14.
- In line 16, we start a loop for each table in our `df` and, from line 18 to 22, create & run queries to collect data from these tables.
- Note that, in line 17, we start a try loop. This is due to the fact that some tables will not have an ORGPERMID column, and the code will fail. In line with this, we create an except statement from line 27 to 29.
- Finally, from line 23 to 36, we make sure that if we receive data from our SQL query, we display this data with `print` statements.

We can now see from the `PERMINSTRREF` table that the company probably is "ENI" (as highligthed in red boxes in the image above); but let's check. With this code below, I only output the table and columns names for which "ENI" is found, fully or partially:

```
    query = f"""
        SELECT TABLE_NAME
        FROM SHAREDB_SFI_MVP.INFORMATION_SCHEMA.TABLES
        WHERE TABLE_NAME ILIKE '%Perm%'
        """
    df = session.sql(query).toPandas()
    
    for table in df.TABLE_NAME:
        try:
            query = f"""
                SELECT * FROM SHAREDB_SFI_MVP.DBO.{table}
                WHERE ORGPERMID = '4295875633'
                """
            _df = session.sql(query).toPandas()
            columns_with_name = [col for col in _df.columns if _df[col].astype(str).str.contains('ENI', case=False, na=False).any()]
            if len(_df) > 0 and len(columns_with_name) > 0:
                print("\n")
                print(f"Table {table}'s columns where 'ENI' appears:", columns_with_name)
        except:
            print("\n")
            print(f"{table} didn't have a 'ORGPERMID' column")
```

![Image](./Docs/PythonWrkSht4.png)

I like the sound of 'LEGALNAME'; let's look into it. First let's return to our SQL worksheet:

![Image](./Docs/PythonWrkSht5.png)

and let's use

```
SELECT *
FROM SHAREDB_SFI_MVP.DBO.PERMORGINFO
LIMIT 10;
```

![Image](./Docs/SFBrowsr10.png)

LEGALNAME is indeed quite a lot better; we were right to check all the table and columns names for which "ENI" is found, fully or partially, since it looks like we should focus on table SHAREDB_SFI_MVP.DBO.PERMORGINFO.

While we looked for this OrgId, you saw that data is split in between all the different databases. Looking for the correct database is a little difficult, but once you get a handle of the Python Worksheets, it's easier.

Now, going forward, we are looking for climate data. For this article, I was after Climate data, specifically Green House Gasses Emissions; which is often abbreviated to GHG (i.e.: GHGEmission). Looking into the [myaccount.lseg.com/en/product/quantitative-analytics](https://myaccount.lseg.com/en/product/quantitative-analytics) files related to Climate Data, I found promising information about the SFI_MASTER_MVP.DBO.CLMDataPoint database:

![Image](./Docs/SFBrowsr11.png)

Let's look into SFI_MASTER_MVP.DBO.CLMDataPoint

![Image](./Docs/SFBrowsr12.png)

Here's ORGPERMID again, let's use this. If you look through the documentation as we did to find SFI_MASTER_MVP.DBO.CLMDataPoint, you'll see that there is a table called SFI_MASTER_MVP.DBO.CLMITEM with item names in full; there I found 'ShortTermSet1GHGEmissionPercentageReductionTargeted', which was a good candidate for being the item I'm interested in. We can pick the data in our CLMDataPoint that only relate to this item:

```
SELECT *
FROM SFI_MASTER_MVP.DBO.CLMDataPoint
WHERE ORGPERMID = 4295875633
AND ITEM IN (
    SELECT ITEM
    FROM SFI_MASTER_MVP.DBO.CLMITEM
    WHERE FEEDFIELDNAME = 'ShortTermSet1GHGEmissionPercentageReductionTargeted'
);
```

![Image](./Docs/SFBrowsr13.png)

Let's join our tables to return only the data we're after:

```
SELECT 
    p.ORGPERMID,
    p.LEGALNAME,
    c.FY,
    c.VALUEPRCNT
FROM 
    SHAREDB_SFI_MVP.DBO.PERMORGINFO p
JOIN 
    SFI_MASTER_MVP.DBO.CLMDataPoint c
    ON p.ORGPERMID = c.ORGPERMID
WHERE 
    c.ORGPERMID = 4295875633
    AND c.ITEM IN (
        SELECT ITEM
        FROM SFI_MASTER_MVP.DBO.CLMITEM
        WHERE FEEDFIELDNAME = 'ShortTermSet1GHGEmissionPercentageReductionTargeted'
    );
```

![Image](./Docs/SFBrowsr14.png)

## Advanced steps: SFI Snowflake using Python and Snowpark

We can get inthe meat of the subject now, and use Python to fetch the data we're after in SFI Snowflake to then create the output we're after: investigate Public Companies' Climate targets and whether they are on track to meet them. To do this, we'll try and create a graph exemplifying our investigation results.

We can do everything we did above, but using Python, e.g.: get the tables of SHAREDB_SFI_MVP:

In [8]:
query = f"""
    SELECT TABLE_NAME
    FROM SHAREDB_SFI_MVP.INFORMATION_SCHEMA.TABLES
    WHERE TABLE_NAME ILIKE '%Perm%'
    """

df = g_db_session.sql(query).toPandas()

display(df)

,TABLE_NAME
0,CLMORGPERMAP
1,ESG2ORGPERMAP
2,ESG2ORGPERMAP_CHANGES
3,PERMINSTRINFO
4,PERMINSTRREF
5,PERMORGINFO
6,PERMORGREF
7,PERMQUOTEREF
8,PERMSECMAPX
9,PERMCODE


Look through these tables to find the table we're after:

In [9]:
dfs = {}
for table in df.TABLE_NAME:
    try:
        query = f"""
            SELECT * FROM SHAREDB_SFI_MVP.DBO.{table}
            WHERE ORGPERMID = '4295875633'
            """
        _df = g_db_session.sql(query).toPandas()
        dfs[f"{table}"] = _df
        if len(_df) > 0:
            print(f"{table}")
            display(_df)
    except:
        print(f"{table} didn't have a 'ORGPERMID' column")

CLMORGPERMAP


,ORGPERMID,PERID,PERENDDT
0,4295875633,94,2011-12-31
1,4295875633,438,2020-12-31
2,4295875633,84,2009-12-31
3,4295875633,400,2018-12-31
4,4295875633,92,2010-12-31
5,4295875633,36,2004-12-31
6,4295875633,154,2013-12-31
7,4295875633,58,2006-12-31
8,4295875633,47,2005-12-31
9,4295875633,304,2014-12-31


ESG2ORGPERMAP


,ORGPERMID,PERID,PERENDDT
0,4295875633,84,2009-12-31
1,4295875633,36,2004-12-31
2,4295875633,23,2002-12-31
3,4295875633,92,2010-12-31
4,4295875633,456,2021-12-31
5,4295875633,420,2019-12-31
6,4295875633,319,2015-12-31
7,4295875633,94,2011-12-31
8,4295875633,28,2003-12-31
9,4295875633,347,2016-12-31


PERMINSTRINFO didn't have a 'ORGPERMID' column
PERMINSTRREF


...

PERMORGINFO


,ORGPERMID,ADDRTYPECODE,ISPUBLIC,IPODATE,ISO2CNTRYCODE,LEGALNAME,ADDRLINE1,ADDRLINE2,CITY,STATEPROV,POSTALCODE,PRITRBCINDPERMID,CNTRYGEOID
0,4295875633,2,True,1995-11-28,IT,ENI S.P.A.,Piazzale Enrico Mattei 1,,ROMA,ROMA,00144,4294951998,100145
1,4295875633,1,True,1995-11-28,IT,ENI S.P.A.,"Piazza Ezio Vanoni, 1",,SAN DONATO MILANESE,MILANO,20097,4294951998,100145


PERMORGREF


,ORGPERMID,STATUS,TYPECODE,DOMCNTRYPERMID,INCCNTRYPERMID,PRIINSTRPERMID,VALQUOTEPERMID,COMNAME,IMMEDIATEPARENTORGPERMID,ULTIMATEPARENTORGPERMID,PRIESTPERMID,WORLDSCOPECMPID,PRIRPTENTITYCODE,LEI,CIK,LIPPERID,ORGANIZATIONPI
0,4295875633,Act,COM,100145,100145,8590929140,55837437646,Eni SpA,4295875633,4295875633,30064794631,C380C0390,A49DF,BUCRF72VH5RBN7X3VL35,,NaN,31644


PERMQUOTEREF didn't have a 'ORGPERMID' column
PERMSECMAPX didn't have a 'ORGPERMID' column
PERMCODE didn't have a 'ORGPERMID' column
PERMQUOTEINFO didn't have a 'ORGPERMID' column


In [10]:
dfs = {}
for table in df.TABLE_NAME:
    try:
        query = f"""
            SELECT * FROM SHAREDB_SFI_MVP.DBO.{table}
            WHERE ORGPERMID = '4295875633'
            """
        _df = g_db_session.sql(query).toPandas()
        dfs[f"{table}"] = _df
        columns_with_name = [col for col in _df.columns if _df[col].astype(str).str.contains('ENI', case=False, na=False).any()]
        if len(_df) > 0 and len(columns_with_name) > 0:
            print(f"Table {table}'s columns where 'ENI' appears:", columns_with_name)
    except:
        print(f"{table} didn't have a 'ORGPERMID' column")

PERMINSTRINFO didn't have a 'ORGPERMID' column
Table PERMINSTRREF's columns where 'ENI' appears: ['COMNAME']
Table PERMORGINFO's columns where 'ENI' appears: ['LEGALNAME']
Table PERMORGREF's columns where 'ENI' appears: ['COMNAME']
PERMQUOTEREF didn't have a 'ORGPERMID' column
PERMSECMAPX didn't have a 'ORGPERMID' column
PERMCODE didn't have a 'ORGPERMID' column
PERMQUOTEINFO didn't have a 'ORGPERMID' column


We can look for Climate data as we did before too:

In [11]:
query_climate = f"""
    SELECT *
    FROM SFI_MASTER_MVP.DBO.CLMDataPoint
    WHERE ORGPERMID = {OrgPermID}
    AND ITEM IN (
        SELECT ITEM
        FROM SFI_MASTER_MVP.DBO.CLMITEM
        WHERE FEEDFIELDNAME = 'ShortTermSet1GHGEmissionPercentageReductionTargeted'
    );
    """

# Execute the query and get the DataFrame
df_climate1 = g_db_session.sql(query_climate).toPandas()

display(df_climate1)

,ORGPERMID,ITEM,FY,VALUERECDT,UNITPCT,UNITPERMID,ISOCURRCODE,VALUE_,VALUEDATE,VALUEFLG,VALUETEXT,VALUEPRCNT
0,4295875633,212,2022,2024-04-26 01:28:33.617,NaN,NaN,None,NaN,NaT,None,None,5000.0
1,4295875633,212,2021,2024-06-06 14:11:48.107,NaN,NaN,None,NaN,NaT,None,None,5000.0
2,4295875633,212,2020,2024-06-06 13:42:58.953,NaN,NaN,None,NaN,NaT,None,None,5000.0


Let's collect only the data we're after:

In [12]:
df_climate2 = df_climate1[['ORGPERMID', 'FY', 'VALUERECDT', 'VALUEPRCNT']]
df_climate2

,ORGPERMID,FY,VALUERECDT,VALUEPRCNT
0,4295875633,2022,2024-04-26 01:28:33.617,5000.0
1,4295875633,2021,2024-06-06 14:11:48.107,5000.0
2,4295875633,2020,2024-06-06 13:42:58.953,5000.0


Now let's collect company info too, as we did above, in Snowflake's SQL Worksheets:

In [13]:
query_climate = f"""
    SELECT
        c.FY,
        p.ORGPERMID,
        p.LEGALNAME,
        c.VALUEPRCNT
    FROM 
        SHAREDB_SFI_MVP.DBO.PERMORGINFO p
    JOIN 
        SFI_MASTER_MVP.DBO.CLMDataPoint c
        ON p.ORGPERMID = c.ORGPERMID
    WHERE 
        c.ORGPERMID = 4295875633
        AND c.ITEM IN (
            SELECT ITEM
            FROM SFI_MASTER_MVP.DBO.CLMITEM
            WHERE FEEDFIELDNAME = 'ShortTermSet1GHGEmissionPercentageReductionTargeted'
        );
    """

# Execute the query and get the DataFrame
df_climate1 = g_db_session.sql(query_climate).toPandas()

display(df_climate1)

,FY,ORGPERMID,LEGALNAME,VALUEPRCNT
0,2021,4295875633,ENI S.P.A.,5000.0
1,2021,4295875633,ENI S.P.A.,5000.0
2,2022,4295875633,ENI S.P.A.,5000.0
3,2022,4295875633,ENI S.P.A.,5000.0
4,2020,4295875633,ENI S.P.A.,5000.0
5,2020,4295875633,ENI S.P.A.,5000.0


Note now that we look for companies using their LEGALNAME with the below (note how, in line 13, we're looking for the name of a company, not the OrgId any more):

In [14]:
query_climate = f"""
    SELECT
        c.FY,
        p.ORGPERMID,
        p.LEGALNAME,
        c.VALUEPRCNT
    FROM 
        SHAREDB_SFI_MVP.DBO.PERMORGINFO p
    JOIN 
        SFI_MASTER_MVP.DBO.CLMDataPoint c
        ON p.ORGPERMID = c.ORGPERMID
    WHERE 
        p.LEGALNAME ILIKE '%Total%'
        AND c.ITEM IN (
            SELECT ITEM
            FROM SFI_MASTER_MVP.DBO.CLMITEM
            WHERE FEEDFIELDNAME = 'ShortTermSet1GHGEmissionPercentageReductionTargeted'
        );
    """

# Execute the query and get the DataFrame
df_climate1 = g_db_session.sql(query_climate).toPandas()
# df_climate1.drop_duplicates(subset=['FY']).sort_values(by=['FY'])
display(df_climate1)

,FY,ORGPERMID,LEGALNAME,VALUEPRCNT
0,2022,5001170594,TotalEnergies SE,1500.0
1,2022,5001170594,TotalEnergies SE,1500.0


Now I'm after another field of interest that I think ought to be in the same table, let's JOIN with SFI_MASTER_MVP.DBO.CLMITEM for all relevent FEEDFIELDNAMEs:

In [15]:
query_climate = f"""
    SELECT *
    FROM 
        SHAREDB_SFI_MVP.DBO.PERMORGINFO p
    JOIN 
        SFI_MASTER_MVP.DBO.CLMDataPoint c
        ON p.ORGPERMID = c.ORGPERMID
    JOIN
        SFI_MASTER_MVP.DBO.CLMITEM k
        ON c.ITEM = k.ITEM
    WHERE 
        p.LEGALNAME ILIKE '%TotalEnergies%'
        AND c.ITEM IN (
            SELECT ITEM
            FROM SFI_MASTER_MVP.DBO.CLMITEM
        );
    """

# Execute the query and get the DataFrame
df_climate2 = g_db_session.sql(query_climate).toPandas()
# df_climate2.drop_duplicates(subset=['FY']).sort_values(by=['FY'])
display(df_climate2)

,ORGPERMID,ADDRTYPECODE,ISPUBLIC,IPODATE,ISO2CNTRYCODE,LEGALNAME,ADDRLINE1,ADDRLINE2,CITY,STATEPROV,...,TITLE,DESCRIPTION,DATATYPECODE,TABLECODE,HEIRARCHYCODE,UNITS,POLARITY,INCLUDEDINSCORES,PILLARITEM,CATITEM
0,5000180598,2,True,2015-05-29,MA,TotalEnergies Marketing Maroc SA,"146, boulevard Zerktouni",,CASABLANCA,GRAND CASABLANCA,...,Short Term Set 1 Upstream Fuel and Energy Rela...,Short-Term Targets Set 1: Does the company rep...,1,3,1,Y/N,Not applicable,None,4,1
1,5001170594,1,True,1973-09-26,FR,TotalEnergies SE,2 Place Jean Millier,Paris la Defense cedex,PARIS,ILE-DE-FRANCE,...,Long Term Set 2 Downstream Investments,Long-Term Targets Set 2: Does the company repo...,1,3,1,Y/N,Not applicable,None,4,1
2,5001170594,2,True,1973-09-26,FR,TotalEnergies SE,La Defense 6,2 Pl Jean Millier,COURBEVOIE,ILE-DE-FRANCE,...,Long Term Set 2 Downstream Investments,Long-Term Targets Set 2: Does the company repo...,1,3,1,Y/N,Not applicable,None,4,1
3,5001170594,2,True,1973-09-26,FR,TotalEnergies SE,La Defense 6,2 Pl Jean Millier,COURBEVOIE,ILE-DE-FRANCE,...,Short Term Set 2 GHG Emission Percentage Reduc...,Short-Term Targets Set 2: Short-term GHG emiss...,3,3,1,Percentage,Positive,None,4,1
4,5001170594,2,True,1973-09-26,FR,TotalEnergies SE,La Defense 6,2 Pl Jean Millier,COURBEVOIE,ILE-DE-FRANCE,...,Green Capex,Does the company disclose the current share or...,1,3,1,Y/N,Positive,None,4,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2721,5001170594,2,True,1973-09-26,FR,TotalEnergies SE,La Defense 6,2 Pl Jean Millier,COURBEVOIE,ILE-DE-FRANCE,...,Energy Purchased Direct,Direct energy purchased in gigajoules. - direc...,3,3,1,Gigajoules,Negative,None,4,3
2722,5001170594,1,True,1973-09-26,FR,TotalEnergies SE,2 Place Jean Millier,Paris la Defense cedex,PARIS,ILE-DE-FRANCE,...,Energy Purchased Direct,Direct energy purchased in gigajoules. - direc...,3,3,1,Gigajoules,Negative,None,4,3
2723,5001170594,2,True,1973-09-26,FR,TotalEnergies SE,La Defense 6,2 Pl Jean Millier,COURBEVOIE,ILE-DE-FRANCE,...,Energy Purchased Direct,Direct energy purchased in gigajoules. - direc...,3,3,1,Gigajoules,Negative,None,4,3
2724,5001170594,1,True,1973-09-26,FR,TotalEnergies SE,2 Place Jean Millier,Paris la Defense cedex,PARIS,ILE-DE-FRANCE,...,Short Term Set 2 Upstream Other,Short-Term Targets Set 2: Does the company rep...,1,3,1,Y/N,Not applicable,None,4,1


I can't see all the column names like this, let's look at the Transpose:

In [16]:
df_climate2.T

,0,1,2,3,4,5,6,7,8,9,...,2716,2717,2718,2719,2720,2721,2722,2723,2724,2725
ORGPERMID,5000180598,5001170594,5001170594,5001170594,5001170594,5001170594,5001170594,5001170594,5001170594,5000180598,...,5001170594,5001170594,5001170594,5001170594,5000180598,5001170594,5001170594,5001170594,5001170594,5001170594
ADDRTYPECODE,2,1,2,2,2,2,2,2,1,1,...,2,2,1,2,2,2,1,2,1,1
ISPUBLIC,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
IPODATE,2015-05-29 00:00:00,1973-09-26 00:00:00,1973-09-26 00:00:00,1973-09-26 00:00:00,1973-09-26 00:00:00,1973-09-26 00:00:00,1973-09-26 00:00:00,1973-09-26 00:00:00,1973-09-26 00:00:00,2015-05-29 00:00:00,...,1973-09-26 00:00:00,1973-09-26 00:00:00,1973-09-26 00:00:00,1973-09-26 00:00:00,2015-05-29 00:00:00,1973-09-26 00:00:00,1973-09-26 00:00:00,1973-09-26 00:00:00,1973-09-26 00:00:00,1973-09-26 00:00:00
ISO2CNTRYCODE,MA,FR,FR,FR,FR,FR,FR,FR,FR,MA,...,FR,FR,FR,FR,MA,FR,FR,FR,FR,FR
LEGALNAME,TotalEnergies Marketing Maroc SA,TotalEnergies SE,TotalEnergies SE,TotalEnergies SE,TotalEnergies SE,TotalEnergies SE,TotalEnergies SE,TotalEnergies SE,TotalEnergies SE,TotalEnergies Marketing Maroc SA,...,TotalEnergies SE,TotalEnergies SE,TotalEnergies SE,TotalEnergies SE,TotalEnergies Marketing Maroc SA,TotalEnergies SE,TotalEnergies SE,TotalEnergies SE,TotalEnergies SE,TotalEnergies SE
ADDRLINE1,"146, boulevard Zerktouni",2 Place Jean Millier,La Defense 6,La Defense 6,La Defense 6,La Defense 6,La Defense 6,La Defense 6,2 Place Jean Millier,"146, boulevard Zerktouni",...,La Defense 6,La Defense 6,2 Place Jean Millier,La Defense 6,"146, boulevard Zerktouni",La Defense 6,2 Place Jean Millier,La Defense 6,2 Place Jean Millier,2 Place Jean Millier
ADDRLINE2,,Paris la Defense cedex,2 Pl Jean Millier,2 Pl Jean Millier,2 Pl Jean Millier,2 Pl Jean Millier,2 Pl Jean Millier,2 Pl Jean Millier,Paris la Defense cedex,,...,2 Pl Jean Millier,2 Pl Jean Millier,Paris la Defense cedex,2 Pl Jean Millier,,2 Pl Jean Millier,Paris la Defense cedex,2 Pl Jean Millier,Paris la Defense cedex,Paris la Defense cedex
CITY,CASABLANCA,PARIS,COURBEVOIE,COURBEVOIE,COURBEVOIE,COURBEVOIE,COURBEVOIE,COURBEVOIE,PARIS,CASABLANCA,...,COURBEVOIE,COURBEVOIE,PARIS,COURBEVOIE,CASABLANCA,COURBEVOIE,PARIS,COURBEVOIE,PARIS,PARIS
STATEPROV,GRAND CASABLANCA,ILE-DE-FRANCE,ILE-DE-FRANCE,ILE-DE-FRANCE,ILE-DE-FRANCE,ILE-DE-FRANCE,ILE-DE-FRANCE,ILE-DE-FRANCE,ILE-DE-FRANCE,GRAND CASABLANCA,...,ILE-DE-FRANCE,ILE-DE-FRANCE,ILE-DE-FRANCE,ILE-DE-FRANCE,GRAND CASABLANCA,ILE-DE-FRANCE,ILE-DE-FRANCE,ILE-DE-FRANCE,ILE-DE-FRANCE,ILE-DE-FRANCE


Looks like there are many duplicates; let's remove them, along with rows where ITEM and VALUEPRCNT are empty:

In [17]:
df_climate2.drop_duplicates().sort_values(by=['FY']).dropna(subset=['ITEM', 'VALUEPRCNT']).T

,2237,8,904,1067,1893,1068,1574,379,2578,2236,...,1139,1398,1310,1454,1892,2539,2065,1705,743,742
ORGPERMID,5001170594,5001170594,5001170594,5001170594,5001170594,5001170594,5000180598,5001170594,5000180598,5001170594,...,5001170594,5001170594,5001170594,5001170594,5000180598,5000180598,5000180598,5000180598,5000180598,5000180598
ADDRTYPECODE,2,1,1,2,2,1,1,1,2,2,...,2,1,1,1,1,2,2,1,2,1
ISPUBLIC,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
IPODATE,1973-09-26 00:00:00,1973-09-26 00:00:00,1973-09-26 00:00:00,1973-09-26 00:00:00,1973-09-26 00:00:00,1973-09-26 00:00:00,2015-05-29 00:00:00,1973-09-26 00:00:00,2015-05-29 00:00:00,1973-09-26 00:00:00,...,1973-09-26 00:00:00,1973-09-26 00:00:00,1973-09-26 00:00:00,1973-09-26 00:00:00,2015-05-29 00:00:00,2015-05-29 00:00:00,2015-05-29 00:00:00,2015-05-29 00:00:00,2015-05-29 00:00:00,2015-05-29 00:00:00
ISO2CNTRYCODE,FR,FR,FR,FR,FR,FR,MA,FR,MA,FR,...,FR,FR,FR,FR,MA,MA,MA,MA,MA,MA
LEGALNAME,TotalEnergies SE,TotalEnergies SE,TotalEnergies SE,TotalEnergies SE,TotalEnergies SE,TotalEnergies SE,TotalEnergies Marketing Maroc SA,TotalEnergies SE,TotalEnergies Marketing Maroc SA,TotalEnergies SE,...,TotalEnergies SE,TotalEnergies SE,TotalEnergies SE,TotalEnergies SE,TotalEnergies Marketing Maroc SA,TotalEnergies Marketing Maroc SA,TotalEnergies Marketing Maroc SA,TotalEnergies Marketing Maroc SA,TotalEnergies Marketing Maroc SA,TotalEnergies Marketing Maroc SA
ADDRLINE1,La Defense 6,2 Place Jean Millier,2 Place Jean Millier,La Defense 6,La Defense 6,2 Place Jean Millier,"146, boulevard Zerktouni",2 Place Jean Millier,"146, boulevard Zerktouni",La Defense 6,...,La Defense 6,2 Place Jean Millier,2 Place Jean Millier,2 Place Jean Millier,"146, boulevard Zerktouni","146, boulevard Zerktouni","146, boulevard Zerktouni","146, boulevard Zerktouni","146, boulevard Zerktouni","146, boulevard Zerktouni"
ADDRLINE2,2 Pl Jean Millier,Paris la Defense cedex,Paris la Defense cedex,2 Pl Jean Millier,2 Pl Jean Millier,Paris la Defense cedex,,Paris la Defense cedex,,2 Pl Jean Millier,...,2 Pl Jean Millier,Paris la Defense cedex,Paris la Defense cedex,Paris la Defense cedex,,,,,,
CITY,COURBEVOIE,PARIS,PARIS,COURBEVOIE,COURBEVOIE,PARIS,CASABLANCA,PARIS,CASABLANCA,COURBEVOIE,...,COURBEVOIE,PARIS,PARIS,PARIS,CASABLANCA,CASABLANCA,CASABLANCA,CASABLANCA,CASABLANCA,CASABLANCA
STATEPROV,ILE-DE-FRANCE,ILE-DE-FRANCE,ILE-DE-FRANCE,ILE-DE-FRANCE,ILE-DE-FRANCE,ILE-DE-FRANCE,GRAND CASABLANCA,ILE-DE-FRANCE,GRAND CASABLANCA,ILE-DE-FRANCE,...,ILE-DE-FRANCE,ILE-DE-FRANCE,ILE-DE-FRANCE,ILE-DE-FRANCE,GRAND CASABLANCA,GRAND CASABLANCA,GRAND CASABLANCA,GRAND CASABLANCA,GRAND CASABLANCA,GRAND CASABLANCA


### 'CO2e' and 'CO2e per Tonne of Revenue'

FEEDFIELDNAME ending with 'GHGEmissionCoveredbyTarget' show-case the amount that we are focussing on which we plan to reduce by a certain percentage, i.e.: if we reduce emissions by 50% on a target of 30%, in total we reduced by 15%, here the 'GHGEmissionCoveredbyTarget' is 30% and 'PercentageReductionTargeted' is 50% (such that 'PercentageReductionTargeted'  is the amount by which it reduced).

This is all relative to the Scope; i.e.: in the example above, we reduced, in total, our 'Scope x' emissions by 15% for x = {1, 2, 3, 1&2, 1&2&3}.

To make things harder, this is all relative, also, to the value a company decides to focus on. In this article, we only focus on 2: 'CO2e' (i.e.: Tons Carbon Dioxide emissions) and 'CO2e per Tonne of Revenue' (i.e.: the Tons Carbon Dioxide emissions per million of revenue, say per 1 000 000 $ or 1 000 000 € of revenue).

We are only interested in 2 VALUETEXT, 'CO2' and 'CO2 Per Tonne of Revenue'. I happen to know that Microsoft has all types, so it's a good use case for now that encapsulates all permutations both in terms of VALUETEXT and Scope (we already know what columns are available in these tables as per the dataframe above, so let's only pick the ones we're after in lines 3 to 8):

In [18]:
query_climate = f"""
    SELECT
        c.FY, p.LEGALNAME, p.ORGPERMID, c.VALUEPRCNT,
        k.ITEM, c.VALUE_, k.INCLUDEDINSCORES,
        k.POLARITY, k.UNITS, k.TABLECODE,
        k.DATATYPECODE, k.DESCRIPTION,
        k.TITLE, k.FEEDFIELDNAME, c.VALUEPRCNT,
        c.VALUETEXT, c.VALUEFLG, c.VALUEDATE
    FROM 
        SHAREDB_SFI_MVP.DBO.PERMORGINFO p
    JOIN 
        SFI_MASTER_MVP.DBO.CLMDataPoint c
        ON p.ORGPERMID = c.ORGPERMID
    JOIN
        SFI_MASTER_MVP.DBO.CLMITEM k
        ON c.ITEM = k.ITEM
    WHERE 
        p.LEGALNAME ILIKE '%Microsoft%'
        AND c.ITEM IN (
            SELECT ITEM
            FROM SFI_MASTER_MVP.DBO.CLMITEM
            WHERE (VALUETEXT = 'CO2' OR VALUETEXT = 'CO2 Per Tonne of Revenue') -- This line makes sure we only have 'CO2' OR 'CO2 Per Tonne of Revenue' publishing companies
        );
    """
df_climate3 = g_db_session.sql(query_climate).toPandas()
display(df_climate3.T)

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
FY,2020,2020,2022,2022,2021,2021,2023,2023,2019,2019,...,2021,2021,2020,2020,2021,2021,2022,2022,2022,2022
LEGALNAME,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,...,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION,MICROSOFT CORPORATION
ORGPERMID,4295907168,4295907168,4295907168,4295907168,4295907168,4295907168,4295907168,4295907168,4295907168,4295907168,...,4295907168,4295907168,4295907168,4295907168,4295907168,4295907168,4295907168,4295907168,4295907168,4295907168
VALUEPRCNT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ITEM,354,354,354,354,354,354,413,413,270,270,...,298,298,326,326,326,326,326,326,382,382
VALUE_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
INCLUDEDINSCORES,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
POLARITY,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,...,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable,Not applicable
UNITS,Name,Name,Name,Name,Name,Name,Reported/CO2/Energy/Median,Reported/CO2/Energy/Median,Name,Name,...,Name,Name,Name,Name,Name,Name,Name,Name,Name,Name
TABLECODE,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,3,3,3,3,3,3


In [19]:
df_climate3.VALUETEXT.unique()

array(['CO2', 'CO2 Per Tonne of Revenue'], dtype=object)

#### 'CO2e'

We can access different datasets related to companies GHG emissions:
- LSEG D&A: Detailed climate data collected, quality controlled and normalized by LSEG from companies' public disclosures, including the auditability back to source documents
- CDP: Climate data as provided by the companies in response to CDP annual survey
- Combined: “Combined” dataset is gathering LSEG data, CDP data and Estimated data based on our robust proprietary multi model estimation approach to provide the best available data for each company. Reported data can be discarded due to additional quality checks (including materiality filter for Scope 3).

For the present use case, as we’re relying on targets disclosed by the companies and collected by our analysts, let’s focus on LSEG D&A reported data.

Depending on the scope of the target (1, 2, 3, 1+2 or 1+2+3) and the unit (CO2e or CO2e per tonne of revenues), we won’t rely on the same datapoint to compute our analysis. The table below shows the direct datapoints to be used (LSEG Workspace code as well as LSEG Data Platform Bulk Field Names):

|                | CO2 'Workspace Code'         | CO2 'Data Platform Bulk Field Names'                | CO2 per Tonne of Revenue 'Workspace Code'       | CO2 per Tonne of Revenue 'Data Platform Bulk Field Names'    |
|----------------|----------------------------|---------------------------------------------------|-----------------------------------------------|------------------------------------------------------------|
| **Scope 1**    | TR.CO2DirectScope1         | CO2EquivalentsEmissionDirectScope1                | TR.AnalyticGHGEmissionsDirectScope1           | GHGEmissionsDirectScope1toRevenue                          |
| **Scope 2**    | TR.CO2IndirectScope2       | CO2EquivalentsEmissionIndirectScope2              | TR.AnalyticGHGEmissionsIndirectScope2         | GHGEmissionsIndirectScope2toRevenue                        |
| **Scope 3**    | TR.CO2IndirectScope3       | CO2EquivalentsEmissionIndirectScope3              | TR.AnalyticCO2IndirectScope3                  | Scope3ToRevenues                                           |
| **Scope 1 and 2**| TR.CO2EmissionTotal       | CO2EquivalentsEmissionTotal                       | TR.AnalyticCO2                                | TotalCO2EquivalentsEmissionsToRevenues                     |
| **Scope 1 and 2 and 3**| TR.TotalCO2EquivalentEmissionsScope1and2and3 | TotalCO2EquivalentEmissionsScope1andScope2andScope3 | TR.AnalyticGHGEmissionsScope1and2and3         | GHGEmissionsScope1andScope2andScope3toRevenue              |

You can also choose to derive all the needed metrics from Scope 1 (CO2EquivalentsEmissionDirectScope1), Scope 2 (CO2EquivalentsEmissionIndirectScope2), Scope 3 (CO2EquivalentsEmissionIndirectScope3)and revenues data.

Let's focus on one use-case for this article; the Reported Direct companies that publish Long Term Set 1 GHG (green house gasses) Emissions; let's also only get the data poits we're interested in, listed in `FEEDFIELDNAME_of_interest`

In [20]:
FEEDFIELDNAME_of_interest = ['CO2EquivalentsEmissionDirectScope1', 'CO2EquivalentsEmissionIndirectScope2', 'CO2EquivalentsEmissionIndirectScope3', 'CO2EquivalentsEmissionTotal', 'TotalCO2EquivalentEmissionsScope1andScope2andScope3']
FEEDFIELDNAME_of_interest_SQL_str = ' OR '.join([f"(FEEDFIELDNAME = '{i}')" for i in FEEDFIELDNAME_of_interest])
FEEDFIELDNAME_of_interest_SQL_str

"(FEEDFIELDNAME = 'CO2EquivalentsEmissionDirectScope1') OR (FEEDFIELDNAME = 'CO2EquivalentsEmissionIndirectScope2') OR (FEEDFIELDNAME = 'CO2EquivalentsEmissionIndirectScope3') OR (FEEDFIELDNAME = 'CO2EquivalentsEmissionTotal') OR (FEEDFIELDNAME = 'TotalCO2EquivalentEmissionsScope1andScope2andScope3')"

In [21]:
query_climate = f"""
    SELECT
        c.FY, p.LEGALNAME, p.ORGPERMID,
        k.ITEM, k.INCLUDEDINSCORES,
        k.POLARITY, k.UNITS, k.TABLECODE,
        k.DATATYPECODE, k.DESCRIPTION,
        k.TITLE, k.FEEDFIELDNAME, c.VALUE_, 
        c.VALUEPRCNT, c.VALUETEXT,
        c.VALUEFLG, c.VALUEDATE
    FROM 
        SHAREDB_SFI_MVP.DBO.PERMORGINFO p
    JOIN 
        SFI_MASTER_MVP.DBO.CLMDataPoint c
        ON p.ORGPERMID = c.ORGPERMID
    JOIN
        SFI_MASTER_MVP.DBO.CLMITEM k
        ON c.ITEM = k.ITEM
    WHERE 
        p.LEGALNAME ILIKE '%Microsoft%'
        AND c.ITEM IN (
            SELECT ITEM
            FROM SFI_MASTER_MVP.DBO.CLMITEM
            WHERE {FEEDFIELDNAME_of_interest_SQL_str}
        );
    """
df_climate4 = g_db_session.sql(query_climate).toPandas()
df_climate4.head(3)

,FY,LEGALNAME,ORGPERMID,ITEM,INCLUDEDINSCORES,POLARITY,UNITS,TABLECODE,DATATYPECODE,DESCRIPTION,TITLE,FEEDFIELDNAME,VALUE_,VALUEPRCNT,VALUETEXT,VALUEFLG,VALUEDATE
0,2010,MICROSOFT CORPORATION,4295907168,12,None,Negative,Tonnes,3,3,Indirect of CO2 and CO2 equivalents emission i...,"CO2 Equivalent Emissions Indirect, Scope 2",CO2EquivalentsEmissionIndirectScope2,1035385.0,NaN,None,None,NaT
1,2010,MICROSOFT CORPORATION,4295907168,12,None,Negative,Tonnes,3,3,Indirect of CO2 and CO2 equivalents emission i...,"CO2 Equivalent Emissions Indirect, Scope 2",CO2EquivalentsEmissionIndirectScope2,1035385.0,NaN,None,None,NaT
2,2013,MICROSOFT CORPORATION,4295907168,11,None,Negative,Tonnes,3,3,Direct of CO2 and CO2 equivalents emission in ...,"CO2 Equivalent Emissions Direct, Scope 1",CO2EquivalentsEmissionDirectScope1,78116.0,NaN,None,None,NaT


In [22]:
df_climate4.pivot_table(
    index='FY',
    columns='FEEDFIELDNAME',
    values=['VALUE_', 'VALUEPRCNT', 'VALUEFLG', 'VALUEDATE', 'VALUETEXT'],
    aggfunc={
        'VALUE_': 'sum',  # for numeric columns, we can use sum, mean, etc.
        'VALUEPRCNT': 'mean',
        'VALUEFLG': 'sum',
        'VALUEDATE': 'first',  # Assuming we want the first value in each group
        'VALUETEXT': 'first'
    }
)

VALUEFLG  \
FEEDFIELDNAME CO2EquivalentsEmissionDirectScope1   
FY                                                 
2005                                           0   
2006                                           0   
2007                                           0   
2008                                         NaN   
2009                                           0   
2010                                           0   
2011                                           0   
2012                                           0   
2013                                           0   
2014                                           0   
2015                                           0   
2016                                           0   
2017                                           0   
2018                                           0   
2019                                           0   
2020                                           0   
2021                                           0   
2022                                           0   

                                                    \
FEEDFIELDNAME CO2EquivalentsEmissionIndirectScope2   
FY                                                   
2005                                             0   
2006                                             0   
2007                                             0   
2008                                           NaN   
2009                                             0   
2010                                             0   
2011                                             0   
2012                                             0   
2013                                             0   
2014                                             0   
2015                                             0   
2016                                             0   
2017                                             0   
2018                                             0   
2019                                             0   
2020                                             0   
2021                                             0   
2022                                             0   

                                                    \
FEEDFIELDNAME CO2EquivalentsEmissionIndirectScope3   
FY                                                   
2005                                           NaN   
2006                                           NaN   
2007                                             0   
2008                                           NaN   
2009                                             0   
2010                                             0   
2011                                             0   
2012                                             0   
2013                                             0   
2014                                             0   
2015                                             0   
2016                                             0   
2017                                             0   
2018                                             0   
2019                                             0   
2020                                             0   
2021                                             0   
2022                                             0   

                                           \
FEEDFIELDNAME CO2EquivalentsEmissionTotal   
FY                                          
2005                                    0   
2006                                    0   
2007                                    0   
2008                                    0   
2009                                    0   
2010                                    0   
2011                                    0   
2012                                    0   
2013                                    0   
2014                                    0   
2015                                    0   
2016                                    0   
20

Clearly, the item of interest here is 'VALUE_':

In [311]:
_df_climate = df_climate4.pivot_table(
    index='FY',
    columns='FEEDFIELDNAME',
    values=['VALUE_'])['VALUE_']
_df_climate

FEEDFIELDNAME,CO2EquivalentsEmissionDirectScope1,CO2EquivalentsEmissionIndirectScope2,CO2EquivalentsEmissionIndirectScope3,CO2EquivalentsEmissionTotal,TotalCO2EquivalentEmissionsScope1andScope2andScope3
FY,,,,,
2005,13185.0,241730.0,NaN,254915.0,NaN
2006,14730.0,137870.0,NaN,152600.0,NaN
2007,15100.0,152480.0,255370.0,167580.0,NaN
2008,NaN,NaN,NaN,1130000.0,NaN
2009,46066.0,799859.0,291888.0,845925.0,NaN
2010,41649.0,1035385.0,289194.0,1077034.0,NaN
2011,47383.0,1144271.0,389017.0,1191654.0,NaN
2012,40848.0,1186492.0,303012.0,1227340.0,NaN
2013,78116.0,1277364.0,5584543.0,1355480.0,NaN


let's add the 'GHGEmissionCoveredbyTarget' and the 'PercentageReductionTargeted' as well as the LongTermSet1GHGEmissionScope, LongTermSet2GHGEmissionScope, ...

In [312]:
query_climate = f"""
    SELECT
        c.FY, p.LEGALNAME, p.ORGPERMID, c.VALUEPRCNT,
        k.ITEM, c.VALUE_, k.INCLUDEDINSCORES,
        k.POLARITY, k.UNITS, k.TABLECODE,
        k.DATATYPECODE, k.DESCRIPTION,
        k.TITLE, k.FEEDFIELDNAME, c.VALUEPRCNT,
        c.VALUETEXT, c.VALUEFLG, c.VALUEDATE
    FROM
        SHAREDB_SFI_MVP.DBO.PERMORGINFO p
    JOIN
        SFI_MASTER_MVP.DBO.CLMDataPoint c
        ON p.ORGPERMID = c.ORGPERMID
    JOIN
        SFI_MASTER_MVP.DBO.CLMITEM k
        ON c.ITEM = k.ITEM
    WHERE
        p.LEGALNAME ILIKE '%Microsoft%'
        AND c.ITEM IN (
            SELECT ITEM
            FROM SFI_MASTER_MVP.DBO.CLMITEM
            WHERE (
                FEEDFIELDNAME ILIKE '%GHGEmissionCoveredbyTarget%')
                OR (FEEDFIELDNAME ILIKE '%PercentageReductionTargeted%')
                OR (FEEDFIELDNAME ILIKE '%LongTermSet%GHGEmissionScope%')
                OR (FEEDFIELDNAME ILIKE '%LongTermSet%GHGEmissionBaseYear%')
                OR (FEEDFIELDNAME ILIKE '%LongTermSet%GHGEmissionTargetYear%')
                OR (FEEDFIELDNAME ILIKE '%CO2EquivalentEmissionsTotalMarketbased%')
        );
    """
df_climate5 = g_db_session.sql(query_climate).toPandas()

Let's keep track of the LEGALNAME

In [313]:
legalName = df_climate5.LEGALNAME.iloc[0]
legalName

'MICROSOFT CORPORATION'

We're are interested in VALUETEXT figures because they let us know which Scopes we focus on for each emission data point.

We also need the base year from which we do our calculation, i.e.: when is the emission reduction calculated from? `FEEDFIELDNAME ILIKE '%LongTermSet%GHGEmissionBaseYear%'` fetches just this info. We can collect it via The VALUE_ values.

We also need the aim year from which we do our calculation, i.e.: when is the emission reduction set to be done by? `FEEDFIELDNAME ILIKE '%LongTermSet%GHGEmissionTargetYear%'` fetches just this info. We can collect it via The VALUE_ values.

The VALUEPRCNT values are the others we're interested in.

Let's concatenate all data in an easy-to-use dataframe `df_climate6`:

In [314]:
df_climate6 = pd.concat(
    [
        df_climate5.pivot_table(
            index='FY',
            columns='FEEDFIELDNAME',
            values=['VALUEPRCNT'],
        ),
        df_climate5.pivot_table(
            index='FY',
            columns='FEEDFIELDNAME',
            values=['VALUETEXT'],
            aggfunc={
                'VALUETEXT': 'first'
            }
        ),
        df_climate5.pivot_table(
            index='FY',
            columns='FEEDFIELDNAME',
            values=['VALUE_'],
        )
    ],
    axis=1)
df_climate6

VALUEPRCNT  \
FEEDFIELDNAME LongTermSet1GHGEmissionPercentageReductionTargeted   
FY                                                                 
2019                                                      7500.0   
2020                                                      7500.0   
2021                                                      7500.0   
2022                                                      7500.0   

                                                                  \
FEEDFIELDNAME LongTermSet1PercentageofGHGEmissionCoveredbyTarget   
FY                                                                 
2019                                                     10000.0   
2020                                                     10000.0   
2021                                                     10000.0   
2022                                                     10000.0   

                                                                  \
FEEDFIELDNAME LongTermSet2GHGEmissionPercentageReductionTargeted   
FY                                                                 
2019                                                      7500.0   
2020                                                      7500.0   
2021                                                      7500.0   
2022                                                      7500.0   

                                                                  \
FEEDFIELDNAME LongTermSet2PercentageofGHGEmissionCoveredbyTarget   
FY                                                                 
2019                                                     10000.0   
2020                                                     10000.0   
2021                                                     10000.0   
2022                                                     10000.0   

                                                                  \
FEEDFIELDNAME LongTermSet3GHGEmissionPercentageReductionTargeted   
FY                                                                 
2019                                                         NaN   
2020                                                      3000.0   
2021                                                      3000.0   
2022                                                      3000.0   

                                                                  \
FEEDFIELDNAME LongTermSet3PercentageofGHGEmissionCoveredbyTarget   
FY                                                                 
2019                                                         NaN   
2020                                                     10000.0   
2021                                                     10000.0   
2022                                                     10000.0   

                                                                  \
FEEDFIELDNAME LongTermSet4GHGEmissionPercentageReductionTargeted   
FY                                                                 
2019                                                         NaN   
2020                                                      5000.0   
2021                                                      5000.0   
2022                                                     10000.0   

                                                                  \
FEEDFIELDNAME LongTermSet4PercentageofGHGEmissionCoveredbyTarget   
FY                                                                 
2019                                                         NaN   
2020                                                     10000.0   
2021                                                     10000.0   
2022                                                         NaN   

                                                                  \
FEEDFIELDNAME LongTermSet5GHGEmissionPercentageReductionTargeted   
FY                                                                 
2019                              

In [315]:
for col in df_climate6.VALUEPRCNT.columns:
    _df_climate[col] = df_climate6.VALUEPRCNT[col].iloc[:,0]

In [316]:
for col in df_climate6.VALUETEXT.columns:
    _df_climate[col] = df_climate6.VALUETEXT[col]

In [317]:
for col in df_climate6.VALUE_.columns:
    _df_climate[col] = df_climate6.VALUE_[col]

In [318]:
_df_climate

FEEDFIELDNAME,CO2EquivalentsEmissionDirectScope1,CO2EquivalentsEmissionIndirectScope2,CO2EquivalentsEmissionIndirectScope3,CO2EquivalentsEmissionTotal,TotalCO2EquivalentEmissionsScope1andScope2andScope3,LongTermSet1GHGEmissionPercentageReductionTargeted,LongTermSet1PercentageofGHGEmissionCoveredbyTarget,LongTermSet2GHGEmissionPercentageReductionTargeted,LongTermSet2PercentageofGHGEmissionCoveredbyTarget,LongTermSet3GHGEmissionPercentageReductionTargeted,...,LongTermSet1GHGEmissionBaseYear,LongTermSet1GHGEmissionTargetYear,LongTermSet2GHGEmissionBaseYear,LongTermSet2GHGEmissionTargetYear,LongTermSet3GHGEmissionBaseYear,LongTermSet3GHGEmissionTargetYear,LongTermSet4GHGEmissionBaseYear,LongTermSet4GHGEmissionTargetYear,LongTermSet5GHGEmissionBaseYear,LongTermSet5GHGEmissionTargetYear
FY,,,,,,,,,,,,,,,,,,,,,
2005,13185.0,241730.0,NaN,254915.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006,14730.0,137870.0,NaN,152600.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007,15100.0,152480.0,255370.0,167580.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008,NaN,NaN,NaN,1130000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,46066.0,799859.0,291888.0,845925.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010,41649.0,1035385.0,289194.0,1077034.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011,47383.0,1144271.0,389017.0,1191654.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012,40848.0,1186492.0,303012.0,1227340.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013,78116.0,1277364.0,5584543.0,1355480.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Some of the values are pre-multiplied by 100 when it's a percentage. I'm personally not a fan of that, so let's scale it back to 100 as a maximum:

In [319]:
for col in _df_climate.columns:
    # Check if column name contains the target substrings
    if ('GHGEmissionPercentageReductionTargeted' in col or 'GHGEmissionCoveredbyTarget' in col):
        # Select the column and check if all numeric values (ignoring NaNs) are greater than 100
        if _df_climate[col].apply(pd.to_numeric, errors='coerce').dropna().gt(100).all():
            # If condition is met, divide the values by 100
            _df_climate[col] = _df_climate[col] / 100
_df_climate

FEEDFIELDNAME,CO2EquivalentsEmissionDirectScope1,CO2EquivalentsEmissionIndirectScope2,CO2EquivalentsEmissionIndirectScope3,CO2EquivalentsEmissionTotal,TotalCO2EquivalentEmissionsScope1andScope2andScope3,LongTermSet1GHGEmissionPercentageReductionTargeted,LongTermSet1PercentageofGHGEmissionCoveredbyTarget,LongTermSet2GHGEmissionPercentageReductionTargeted,LongTermSet2PercentageofGHGEmissionCoveredbyTarget,LongTermSet3GHGEmissionPercentageReductionTargeted,...,LongTermSet1GHGEmissionBaseYear,LongTermSet1GHGEmissionTargetYear,LongTermSet2GHGEmissionBaseYear,LongTermSet2GHGEmissionTargetYear,LongTermSet3GHGEmissionBaseYear,LongTermSet3GHGEmissionTargetYear,LongTermSet4GHGEmissionBaseYear,LongTermSet4GHGEmissionTargetYear,LongTermSet5GHGEmissionBaseYear,LongTermSet5GHGEmissionTargetYear
FY,,,,,,,,,,,,,,,,,,,,,
2005,13185.0,241730.0,NaN,254915.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2006,14730.0,137870.0,NaN,152600.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007,15100.0,152480.0,255370.0,167580.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2008,NaN,NaN,NaN,1130000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,46066.0,799859.0,291888.0,845925.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010,41649.0,1035385.0,289194.0,1077034.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011,47383.0,1144271.0,389017.0,1191654.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012,40848.0,1186492.0,303012.0,1227340.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013,78116.0,1277364.0,5584543.0,1355480.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


For Your information (FYI): CO2EquivalentsEmissionTotal stands for TotalCO2EquivalentEmissionsScope1andScope2. As we can see in the far right of `_df_climate`, Scope1 & Scope2 seems the most popular, surely because that’s where companies have operational control; let's stick with that. Let's also choose 'Set 1' for this article, but you can choose whichever suits your needs.

In [320]:
df_climate = _df_climate[[
    "CO2EquivalentsEmissionDirectScope1",
    "CO2EquivalentsEmissionIndirectScope2",
    "CO2EquivalentsEmissionTotal"]]

In [321]:
for col in ["CO2EquivalentEmissionsTotalMarketbased",
            "LongTermSet1GHGEmissionPercentageReductionTargeted",
            "LongTermSet1PercentageofGHGEmissionCoveredbyTarget",
            "LongTermSet1GHGEmissionBaseYear",
            "LongTermSet1GHGEmissionTargetYear"]:
    df_climate[col] = _df_climate[_df_climate['LongTermSet1GHGEmissionScope'] == "Scope 1 and 2"][col]
df_climate

C:\Users\U6082174\AppData\Local\Temp\ipykernel_4952\1823118552.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\U6082174\AppData\Local\Temp\ipykernel_4952\1823118552.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\U6082174\AppData\Local\Temp\ipykernel_4952\1823118552.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org

FEEDFIELDNAME,CO2EquivalentsEmissionDirectScope1,CO2EquivalentsEmissionIndirectScope2,CO2EquivalentsEmissionTotal,CO2EquivalentEmissionsTotalMarketbased,LongTermSet1GHGEmissionPercentageReductionTargeted,LongTermSet1PercentageofGHGEmissionCoveredbyTarget,LongTermSet1GHGEmissionBaseYear,LongTermSet1GHGEmissionTargetYear
FY,,,,,,,,
2005,13185.0,241730.0,254915.0,NaN,NaN,NaN,NaN,NaN
2006,14730.0,137870.0,152600.0,NaN,NaN,NaN,NaN,NaN
2007,15100.0,152480.0,167580.0,NaN,NaN,NaN,NaN,NaN
2008,NaN,NaN,1130000.0,NaN,NaN,NaN,NaN,NaN
2009,46066.0,799859.0,845925.0,NaN,NaN,NaN,NaN,NaN
2010,41649.0,1035385.0,1077034.0,NaN,NaN,NaN,NaN,NaN
2011,47383.0,1144271.0,1191654.0,NaN,NaN,NaN,NaN,NaN
2012,40848.0,1186492.0,1227340.0,NaN,NaN,NaN,NaN,NaN
2013,78116.0,1277364.0,1355480.0,NaN,NaN,NaN,NaN,NaN


As it happens to be, I picked Microsoft and their 'GHGEmissionCoveredbyTarget' is always 100%. In truth, all companies' isn't necessarily this high, and the lower this figure, the less reliable the results. Therefore, going forward, we'll add a 80% threshold under which we show an error.

In [322]:
if not df_climate['LongTermSet1PercentageofGHGEmissionCoveredbyTarget'].apply(pd.to_numeric, errors='coerce').dropna().gt(80).all():
    print("Be weary of the fact that 'GHGEmissionCoveredbyTarget' is below 80%. The lower this figure, the less reliable the results.")
else:
    print("'GHGEmissionCoveredbyTarget' figures are above 80%. The higher this figure, the more reliable the results.")
    print("In this case, we consider them to be high enough to be considered reliable.")

'GHGEmissionCoveredbyTarget' figures are above 80%. The higher this figure, the more reliable the results.
In this case, we consider them to be high enough to be considered reliable.


So, let's recap; the base emission is `baseEmission` and the target is `targetEmission` to be archeved by 2045:

In [323]:
baseEmissionYear = int(df_climate.LongTermSet1GHGEmissionBaseYear.iloc[-1])
targetEmissionReductionYear = int(df_climate.LongTermSet1GHGEmissionTargetYear.iloc[-1])
baseEmission = _df_climate.loc[baseEmissionYear].CO2EquivalentsEmissionTotal
targetReductionEmission = baseEmission * (df_climate['LongTermSet1GHGEmissionPercentageReductionTargeted'].iloc[-1] / 100) * (df_climate['LongTermSet1PercentageofGHGEmissionCoveredbyTarget'].iloc[-1] / 100)
targetEmission = baseEmission - targetReductionEmission
print(f"baseEmissionYear = {baseEmissionYear} & targetEmissionReductionYear = {targetEmissionReductionYear}")
print(f"baseEmission = {baseEmission} & targetReductionEmission = {targetReductionEmission} & targetEmission = {targetEmission}")

baseEmissionYear = 2013 & targetEmissionReductionYear = 2045
baseEmission = 1355480.0 & targetReductionEmission = 1016610.0 & targetEmission = 338870.0


It's always best to see it all in a pretty graph:

#### Exponential Decay Line Plot Explanation

To create an exponential decay plot, we use the following exponential decay function:
$$
y(t) = y_0 \cdot e^{-k(t - t_0)}
$$

Where:
- $y_0$ is the initial value at the starting point (1,355,480.0 in 2013),
- $ k $ is a constant that controls the rate of decay,
- $ t $ is the year (time),
- $ t_0 $ is the start year (2013),
- $ y(t) $ is the value at year $ t $.

##### Steps:

**Calculate the Decay Constant $ k $**:
   We first calculate the decay constant $ k $ based on the start and end values (1,355,480.0 at 2013 and 338,870.0 at 2045). We use the following formula for exponential decay:

   $$
   k = \frac{-\log\left(\frac{y_{\text{end}}}{y_0}\right)}{(t_{\text{end}} - t_0)}
   $$

   This formula is derived by solving for $ k $ in the exponential decay equation, where $ y_{\text{end}} $ is the value at the end year (2045), $ y_0 $ is the starting value (2013), and $ t_{\text{end}} $ is the end year.

**Generate the Exponential Values**:
   With $ k $ calculated, we use the exponential decay equation to generate values for each year between 2013 and 2045:

   $$
   y(t) = y_0 \cdot e^{-k(t - t_0)}
   $$

   This generates values that decrease exponentially from 1,355,480.0 in 2013 to 338,870.0 in 2045.

In [ ]:
start_year = baseEmissionYear
end_year = targetEmissionReductionYear
start_value = baseEmission
end_value = targetEmission

# Calculate the decay constant 'k' for exponential decay
# We use the formula for exponential decay: y(t) = y0 * exp(-k * (t - t0))
# Solve for 'k' using y(t) at the end point
k = -np.log(end_value / start_value) / (end_year - start_year)

# Generate years and corresponding values using exponential decay
years = np.arange(start_year, end_year + 1)
values = start_value * np.exp(-k * (years - start_year))

# Create the plot
fig = go.Figure()

# Add the line trace
fig.add_trace(
    go.Scatter(
        x=years, y=values, mode='lines',
        name=f'Exponential Decay to target of {int(targetEmission)}'))

# Set plot title and labels
fig.update_layout(
    title=f"{legalName}'s GHG Emissions (Base in {baseEmissionYear} and Target in {targetEmissionReductionYear}) in CO2 Tons",
    xaxis_title="Year",
    yaxis_title="Value",
    template="plotly_dark"  # Optional: You can use different templates, such as 'plotly', 'ggplot2', etc.
)

# Add the dots representing the actual CO2 data from df_clmt0
fig.add_trace(go.Scatter(
    x=df_climate.index, 
    y=df_climate["CO2EquivalentsEmissionTotal"], 
    mode='lines+markers', 
    name='CO2 Equivalents Emission Total',
    marker=dict(color='red', size=8)  # Customize the dot color and size
))

fig.add_trace(go.Scatter(
    x=df_climate.index, 
    y=df_climate["CO2EquivalentEmissionsTotalMarketbased"], 
    mode='lines+markers', 
    name='CO2 Equivalent Emissions Total Marketbased',
    marker=dict(color='grey', size=8)  # Customize the dot color and size
))

# Show the plot
fig.show()

## All together now


In [1]:
## Installing packages/libraries


## pip install snowflake-snowpark-python
## pip install "snowflake-connector-python[pandas]"
from snowflake import snowpark
import pandas as pd
import os # This is a native library that will allow us to collect authentication details from the machine where we're running our code.
import plotly.graph_objects as go # `plotly` and its modules are useful for us to plot graphs
import numpy as np # `numpy` in this case will be useful for us to plot graphs}


## authentication to LSEG's SFI Snowflake services


# Global variable
g_db_session = None

loginUserName = "john.doe@domain.com"

def get_session(user, password):
    global g_db_session
    
    # Establish a connection and assign it to the global session variable
    connection_parameters = { 
        "account": "rddpreprod1.east-us-2.azure",
        "user": user,
        "password": password,
        "warehouse": "SFI_READER_S_WH", 
        "database": "SHAREDB_SFI_MVP",
        "role": "SNO_SNOREFDBD_SFI_READER_ROLE",
        "schema": "DBO",
        "authenticator": "externalbrowser"
    }
    
    g_db_session = snowpark.Session.builder.configs(connection_parameters).create() 

get_session(loginUserName, os.getenv('SSO_PASSWD'))

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://login.microsoftonline.com/287e9f0e-91ec-4cf0-b7a4-c63898072181/saml2?SAMLRequest=nZJLc9owFIX%2Fikdd25aMG7AGyDgYpkx5DRg6zU7YMijIkivJMfn3kXl00kWy6E4jnXu%2Fo3tu%2F%2FFccueVKs2kGADkQeBQkcmcicMAbNOJ2wOONkTkhEtBB%2BCNavA47GtS8grHtTmKNf1TU20c20ho3D4MQK0ElkQzjQUpqcYmw5t4PsOBBzHRmipjceBWkmtmWUdjKuz7TdN4TceT6uAHEEIfRr5VtZJv4AOi%2BppRKWlkJvm95Gz%2F9AkC%2BTBsEVZhCatb4RMT1xF8RdlfRRr%2FSNOVu1puUuDE99%2BNpNB1SdWGqleW0e16djWgrQOu6cHdLJbr8SR5SuLn7XiXjHeeFrIpODnRTJZVbWxrz578guY%2BlwdmBzZNBqA6sfz3MVLFiZOfiwMvz6sjHCuyLOZZuNmL2Xf5a49e0ofmZT6K4gw4u3u8QRvvVOuaTkUbqrFXMAhdFLgIpqiDIcJhx0MBegZOYkNlgphL5V%2FnrQ%2BvZJmSWhZGCs4EvbgMel0aFZC6EaKZG2YFdPddErrZQ6cX9WA3QD3kt9EF4Lo%2B%2BGJEDf9rKH3%2FY4vbOi5sQtNkJTnL3pyJVCUxnweIPHS5YblbXKSYloTxOM8V1doGyblsRooSY7feqJoCf3

In [20]:
def GHGEmissionsBaseVsTargetUsingLSEGSFISnowflake(
        g_db_session,
        companyLegalName = "Microsoft",
        exactCompanyLegalName = False,
    ):

    ## Installing packages/libraries


    ## pip install snowflake-snowpark-python
    ## pip install "snowflake-connector-python[pandas]"
    from snowflake import snowpark
    import pandas as pd
    import plotly.graph_objects as go # `plotly` and its modules are useful for us to plot graphs
    import numpy as np # `numpy` in this case will be useful for us to plot graphs


    ## Collecting and tidying Data from LSEG's SFI service on Snowflake


    if exactCompanyLegalName:
        companyLegalNameToUse = f"= '{companyLegalName}'"
    elif exactCompanyLegalName is not True:
        companyLegalNameToUse = f"ILIKE '%{companyLegalName}%'"

    FeedFieldNameOfInterest = [
        'CO2EquivalentsEmissionDirectScope1',
        'CO2EquivalentsEmissionIndirectScope2',
        'CO2EquivalentsEmissionIndirectScope3',
        'CO2EquivalentsEmissionTotal',
        'TotalCO2EquivalentEmissionsScope1andScope2andScope3']
    FeedFieldNameOfInterestSQLStr = ' OR '.join(
        [f"(FEEDFIELDNAME = '{i}')" for i in FeedFieldNameOfInterest])

    query_climate = f"""
        SELECT
            c.FY, p.LEGALNAME, p.ORGPERMID,
            k.ITEM, k.INCLUDEDINSCORES,
            k.POLARITY, k.UNITS, k.TABLECODE,
            k.DATATYPECODE, k.DESCRIPTION,
            k.TITLE, k.FEEDFIELDNAME, c.VALUE_, 
            c.VALUEPRCNT, c.VALUETEXT,
            c.VALUEFLG, c.VALUEDATE
        FROM 
            SHAREDB_SFI_MVP.DBO.PERMORGINFO p
        JOIN 
            SFI_MASTER_MVP.DBO.CLMDataPoint c
            ON p.ORGPERMID = c.ORGPERMID
        JOIN
            SFI_MASTER_MVP.DBO.CLMITEM k
            ON c.ITEM = k.ITEM
        WHERE 
            p.LEGALNAME {companyLegalNameToUse}
            AND c.ITEM IN (
                SELECT ITEM
                FROM SFI_MASTER_MVP.DBO.CLMITEM
                WHERE {FeedFieldNameOfInterestSQLStr}
            );
        """
    _dfClimateFromSFI = g_db_session.sql(query_climate).toPandas()

    _dfClimate = _dfClimateFromSFI.pivot_table(
        index='FY',
        columns='FEEDFIELDNAME',
        values=['VALUE_'])['VALUE_']

    query_climate = f"""
        SELECT
            c.FY, p.LEGALNAME, p.ORGPERMID, c.VALUEPRCNT,
            k.ITEM, c.VALUE_, k.INCLUDEDINSCORES,
            k.POLARITY, k.UNITS, k.TABLECODE,
            k.DATATYPECODE, k.DESCRIPTION,
            k.TITLE, k.FEEDFIELDNAME, c.VALUEPRCNT,
            c.VALUETEXT, c.VALUEFLG, c.VALUEDATE
        FROM
            SHAREDB_SFI_MVP.DBO.PERMORGINFO p
        JOIN
            SFI_MASTER_MVP.DBO.CLMDataPoint c
            ON p.ORGPERMID = c.ORGPERMID
        JOIN
            SFI_MASTER_MVP.DBO.CLMITEM k
            ON c.ITEM = k.ITEM
        WHERE
            p.LEGALNAME {companyLegalNameToUse}
            AND c.ITEM IN (
                SELECT ITEM
                FROM SFI_MASTER_MVP.DBO.CLMITEM
                WHERE (
                    FEEDFIELDNAME ILIKE '%GHGEmissionCoveredbyTarget%')
                    OR (FEEDFIELDNAME ILIKE '%PercentageReductionTargeted%')
                    OR (FEEDFIELDNAME ILIKE '%LongTermSet%GHGEmissionScope%')
                    OR (FEEDFIELDNAME ILIKE '%LongTermSet%GHGEmissionBaseYear%')
                    OR (FEEDFIELDNAME ILIKE '%LongTermSet%GHGEmissionTargetYear%')
                    OR (FEEDFIELDNAME ILIKE '%CO2EquivalentEmissionsTotalMarketbased%')
            );
        """
    dfClimateFromSFI = g_db_session.sql(query_climate).toPandas()

    legalName = dfClimateFromSFI.LEGALNAME.iloc[0]

    __dfClimate = pd.concat(
        [
            dfClimateFromSFI.pivot_table(
                index='FY',
                columns='FEEDFIELDNAME',
                values=['VALUEPRCNT'],
            ),
            dfClimateFromSFI.pivot_table(
                index='FY',
                columns='FEEDFIELDNAME',
                values=['VALUETEXT'],
                aggfunc={
                    'VALUETEXT': 'first'
                }
            ),
            dfClimateFromSFI.pivot_table(
                index='FY',
                columns='FEEDFIELDNAME',
                values=['VALUE_'],
            )
        ],
        axis=1)
    
    for col in __dfClimate.VALUEPRCNT.columns:
        _dfClimate.loc[:, col] = __dfClimate.VALUEPRCNT[col].iloc[:,0]

    for col in __dfClimate.VALUETEXT.columns:
        _dfClimate.loc[:, col] = __dfClimate.VALUETEXT[col]

    for col in __dfClimate.VALUE_.columns:
        _dfClimate.loc[:, col] = __dfClimate.VALUE_[col]
    
    for col in _dfClimate.columns:
        # Check if column name contains the target substrings
        if ('GHGEmissionPercentageReductionTargeted' in col or 'GHGEmissionCoveredbyTarget' in col):
            # Select the column and check if all numeric values (ignoring NaNs) are greater than 100
            if _dfClimate[col].apply(pd.to_numeric, errors='coerce').dropna().gt(100).all():
                # If condition is met, divide the values by 100
                _dfClimate.loc[:, col] = _dfClimate[col] / 100
    
    dfClimate = _dfClimate.loc[:, [
        "CO2EquivalentsEmissionDirectScope1",
        "CO2EquivalentsEmissionIndirectScope2",
        "CO2EquivalentsEmissionTotal"]]
    
    for col in ["CO2EquivalentEmissionsTotalMarketbased",
                "LongTermSet1GHGEmissionPercentageReductionTargeted",
                "LongTermSet1PercentageofGHGEmissionCoveredbyTarget",
                "LongTermSet1GHGEmissionBaseYear",
                "LongTermSet1GHGEmissionTargetYear"]:
        dfClimate.loc[:, col] = _dfClimate.loc[_dfClimate['LongTermSet1GHGEmissionScope'] == "Scope 1 and 2", col]
    
    if not dfClimate['LongTermSet1PercentageofGHGEmissionCoveredbyTarget'].apply(pd.to_numeric, errors='coerce').dropna().gt(80).all():
        print("Be weary of the fact that 'GHGEmissionCoveredbyTarget' is below 80%. The lower this figure, the less reliable the results.")
    else:
        print("'GHGEmissionCoveredbyTarget' figures are above 80%. The higher this figure, the more reliable the results.")
        print("In this case, we consider them to be high enough to be considered reliable.")



    ## Ploting Graph

    
    baseEmissionYear = int(dfClimate.LongTermSet1GHGEmissionBaseYear.dropna().iloc[-1])
    targetEmissionReductionYear = int(dfClimate.LongTermSet1GHGEmissionTargetYear.dropna().iloc[-1])
    baseEmission = dfClimate.loc[baseEmissionYear].CO2EquivalentsEmissionTotal
    targetReductionEmission = baseEmission * (dfClimate['LongTermSet1GHGEmissionPercentageReductionTargeted'].dropna().iloc[-1] / 100) * (dfClimate['LongTermSet1PercentageofGHGEmissionCoveredbyTarget'].dropna().iloc[-1] / 100)
    targetEmission = baseEmission - targetReductionEmission
    print(f"baseEmissionYear = {baseEmissionYear} & targetEmissionReductionYear = {targetEmissionReductionYear}")
    print(f"baseEmission = {baseEmission} & targetReductionEmission = {targetReductionEmission} & targetEmission = {targetEmission}")

    if targetReductionEmission == np.nan:
        print(f"The targetReductionEmission is {targetReductionEmission}. Please note that there seem to be no targetReductionEmission data available for this company.")

    start_year = baseEmissionYear
    end_year = targetEmissionReductionYear
    start_value = baseEmission
    end_value = targetEmission

        # Calculate the decay constant 'k' for exponential decay
    # We use the formula for exponential decay: y(t) = y0 * exp(-k * (t - t0))
    # Solve for 'k' using y(t) at the end point
    if end_value / start_value == 0:
        k = -np.log(0.01) / (end_year - start_year)  # Use a small value instead of zero to avoid log(0) issue
    else:
        k = -np.log(end_value / start_value) / (end_year - start_year)

    # Generate years and corresponding values using exponential decay
    years = np.arange(start_year, end_year + 1)
    values = start_value * np.exp(-k * (years - start_year))

    # Create the plot
    fig = go.Figure()

    # Add the line trace
    fig.add_trace(
        go.Scatter(
            x=years, y=values, mode='lines',
            name=f'Exponential Decay to target of {int(targetEmission)}'))

    # Set plot title and labels
    fig.update_layout(
        title=f"{legalName}'s GHG Emissions (Base in {baseEmissionYear} and Target in {targetEmissionReductionYear}) in CO2 Tons",
        xaxis_title="Year",
        yaxis_title="Value",
        template="plotly_dark"  # Optional: You can use different templates, such as 'plotly', 'ggplot2', etc.
    )

    # Add the dots representing the actual CO2 data from dfClimate
    fig.add_trace(go.Scatter(
        x=dfClimate.index, 
        y=dfClimate["CO2EquivalentsEmissionTotal"], 
        mode='lines+markers', 
        name='CO2 Equivalents Emission Total',
        marker=dict(color='red', size=8)  # Customize the dot color and size
    ))

    fig.add_trace(go.Scatter(
        x=dfClimate.index, 
        y=dfClimate["CO2EquivalentEmissionsTotalMarketbased"], 
        mode='lines+markers', 
        name='CO2 Equivalent Emissions Total Marketbased',
        marker=dict(color='grey', size=8)  # Customize the dot color and size
    ))

    # Show the plot
    fig.show()
    
    return {'legalName': legalName,
            'baseEmissionYear': baseEmissionYear,
            'targetEmissionReductionYear': targetEmissionReductionYear,
            'baseEmission': baseEmission,
            'targetReductionEmission': targetReductionEmission,
            'targetEmission': targetEmission,
            'exponentialDecayValues': values,
            'dfClimate': dfClimate}

In [35]:
dfDict = GHGEmissionsBaseVsTargetUsingLSEGSFISnowflake(
        g_db_session=g_db_session,
        companyLegalName="Shell",
        exactCompanyLegalName=False)

'GHGEmissionCoveredbyTarget' figures are above 80%. The higher this figure, the more reliable the results.
In this case, we consider them to be high enough to be considered reliable.
baseEmissionYear = 2016 & targetEmissionReductionYear = 2050
baseEmission = 83000000.0 & targetReductionEmission = 83000000.0 & targetEmission = 0.0


In [36]:
for i in dfDict.keys():
    print(i)
    display(dfDict[i])

legalName


'SHELL PLC'

baseEmissionYear


2016

targetEmissionReductionYear


...

0.0

exponentialDecayValues


array([83000000.        , 72486071.47777599, 63303982.62989458,
       55285024.21371602, 48281858.03380718, 42165809.78937808,
       36824504.84712248, 32159803.49978589, 28085997.77345435,
       24528236.65218397, 21421150.78547003, 18707651.40929877,
       16337881.41247888, 14268299.27542617, 12460879.04993784,
       10882411.68059899,  9503894.83048774,  8300000.        ,
        7248607.1477776 ,  6330398.26298946,  5528502.4213716 ,
        4828185.80338072,  4216580.97893781,  3682450.48471225,
        3215980.34997859,  2808599.77734544,  2452823.6652184 ,
        2142115.078547  ,  1870765.14092988,  1633788.14124789,
        1426829.92754262,  1246087.90499378,  1088241.1680599 ,
         950389.48304877,   830000.        ])

dfClimate


FEEDFIELDNAME,CO2EquivalentsEmissionDirectScope1,CO2EquivalentsEmissionIndirectScope2,CO2EquivalentsEmissionTotal,CO2EquivalentEmissionsTotalMarketbased,LongTermSet1GHGEmissionPercentageReductionTargeted,LongTermSet1PercentageofGHGEmissionCoveredbyTarget,LongTermSet1GHGEmissionBaseYear,LongTermSet1GHGEmissionTargetYear
FY,,,,,,,,
2002,106000000.0,NaN,106000000.0,NaN,NaN,NaN,NaN,NaN
2003,112000000.0,NaN,112000000.0,NaN,NaN,NaN,NaN,NaN
2004,112000000.0,NaN,112000000.0,NaN,NaN,NaN,NaN,NaN
2005,105000000.0,NaN,105000000.0,NaN,NaN,NaN,NaN,NaN
2006,98000000.0,NaN,98000000.0,NaN,NaN,NaN,NaN,NaN
2007,82000000.0,NaN,82000000.0,NaN,NaN,NaN,NaN,NaN
2008,75000000.0,NaN,75000000.0,NaN,NaN,NaN,NaN,NaN
2009,69000000.0,9000000.0,78000000.0,NaN,NaN,NaN,NaN,NaN
2010,76000000.0,9000000.0,85000000.0,NaN,NaN,NaN,NaN,NaN


# Conclusion

As you can see, once you know the name of the fields you are after, it becomes a simple matter of mix and match which SQL, along with Snowflake, can do for you! The code above is not fool-proof, and is not to be used in live environment, but it's a great example of how one can created automated processes allowing for anyone to gather insight behind SFI data and output useful graphics.